<a href="https://colab.research.google.com/github/cemyyldz/pdf-copy-project/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Ortam Kurulumu ve Kütüphane Yükleme
Bu bölümde, projenin temel ve destekleyici tüm kütüphaneleri kurulur.

In [17]:
# Ortam Kurulumu: Gerekli ve Destekleyici Kütüphaneler
print("Gerekli ve destekleyici kütüphaneler kuruluyor...")
!pip install PyPDF2 pdfminer.six reportlab pdfplumber Pillow
print("Kurulum tamamlandı. Çalışma zamanının yeniden başlatılması gerekebilir.")

# Gerekli tüm modülleri içe aktarma
import os
import shutil
from PyPDF2 import PdfReader
from PIL import Image
import io
import re
import pdfplumber

# pdfminer ve ReportLab içe aktarımları
from pdfminer.layout import LTTextContainer, LTTextLine, LTChar, LTLine, LTRect
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LAParams
from reportlab.pdfgen import canvas
from reportlab.lib.colors import black
from reportlab.pdfbase.pdfmetrics import stringWidth

# Sabitler
PDF_INPUT_FILE = "file.pdf"
PDF_OUTPUT_FILE = "output.pdf"

Gerekli ve destekleyici kütüphaneler kuruluyor...
Kurulum tamamlandı. Çalışma zamanının yeniden başlatılması gerekebilir.


2. Aşama: Yapısal Veri Çıkarımı (Metin, Şekil ve Görsel)Bu aşamada, PDF'ten metin koordinatları (pdfminer.six) ve görseller (pdfplumber) dahil olmak üzere tüm yapısal veriler tek bir project_data yapısına çıkarılır. Görsel konumlandırma sorunlarını çözmek için y ekseni düzeltmeleri bu kısımda yapılır.

In [18]:
# Global veri yapısı ve sayfa boyutları
# PDF içerisinden çıkarılan tüm verileri saklamak için bir sözlük oluşturuyoruz.
# 'text_blocks' -> metin blokları, 'lines' -> çizgiler, 'rects' -> dikdörtgenler, 'images' -> görseller.
project_data = {'text_blocks': [], 'lines': [], 'rects': [], 'images': []}

# Sayfa genişliği ve yüksekliği global değişkenler olarak tanımlanıyor.
page_w, page_h = 0, 0

# Font Eşleştirme Sözlüğü
# PDF'deki font isimlerini ReportLab'ta kullanılacak standart font isimleriyle eşleştiriyoruz.
FONT_MAPPING = {
    'REGULAR': 'Helvetica', 'PRO-REGULAR': 'Helvetica',
    'BOLD': 'Helvetica-Bold', 'PRO-BOLD': 'Helvetica-Bold',
    'ITALIC': 'Helvetica-Oblique', 'PRO-ITALIC': 'Helvetica-Oblique',
    'ARIALMT': 'Helvetica', 'TIMESNEWROMAN': 'Times-Roman', 'COURIER': 'Courier',
}

def get_page_dimensions(file_path):
    """PyPDF2 ile sayfa boyutlarını alır."""
    global page_w, page_h
    try:
        # PDF dosyasını oku
        reader = PdfReader(file_path)
        # İlk sayfayı seç
        page = reader.pages[0]
        # Sayfanın mediabox'ını al ve genişlik-yükseklik değerlerini ata
        media_box = page.mediabox
        page_w, page_h = float(media_box.width), float(media_box.height)
        return True
    except Exception as e:
        # Hata oluşursa False döndür
        return False

def extract_all_structural_elements(file_path):
    """pdfminer.six ile metin, çizgi ve dikdörtgen verilerini çıkarır."""
    # PDF kaynak yöneticisi ve parametreleri ayarla
    rsrcmgr = PDFResourceManager()
    laparams = LAParams(line_overlap=0.5, char_margin=2.0, word_margin=0.1, line_margin=0.5, boxes_flow=0.5, detect_vertical=False)
    device = PDFPageAggregator(rsrcmgr, laparams=laparams)
    interpreter = PDFPageInterpreter(rsrcmgr, device)

    try:
        with open(file_path, 'rb') as fp:
            # PDF'deki her sayfa için işlemi tekrarla
            for page in PDFPage.get_pages(fp):
                interpreter.process_page(page)
                layout = device.get_result()  # Sayfanın tüm yapısını al
                for element in layout:
                    # Metin konteyneri varsa
                    if isinstance(element, LTTextContainer):
                        for text_line in element:
                            if isinstance(text_line, LTTextLine):
                                current_text = ""
                                last_char_data = None
                                for character in text_line:
                                    if isinstance(character, LTChar):
                                        current_text += character.get_text()  # Metni topla
                                        if last_char_data is None:
                                            # Font ve konum bilgilerini al
                                            last_char_data = {'x': character.bbox[0], 'y': character.bbox[1], 'font': character.fontname, 'size': character.size}
                                if current_text and last_char_data:
                                    # Metin ve font bilgisini proje veri yapısına ekle
                                    data = last_char_data
                                    data['text'] = current_text
                                    project_data['text_blocks'].append(data)
                    # Çizgi elemanı varsa
                    elif isinstance(element, LTLine):
                        project_data['lines'].append({'x0': element.bbox[0], 'y0': element.bbox[1], 'x1': element.bbox[2], 'y1': element.bbox[3], 'width': element.linewidth })
                    # Dikdörtgen elemanı varsa
                    elif isinstance(element, LTRect):
                        project_data['rects'].append({'x0': element.bbox[0], 'y0': element.bbox[1], 'x1': element.bbox[2], 'y1': element.bbox[3], 'width': element.linewidth })
    except Exception as e:
        print(f"pdfminer.six ile veri çıkarılırken hata: {e}")

def extract_images_with_pdfplumber(file_path):
    """pdfplumber ile görselleri çıkarır, ReportLab için dikey çevirir ve Y koordinatını hesaplar."""

    # Geçici görsellerin saklanacağı klasörü oluştur
    temp_dir = 'temp_images'
    os.makedirs(temp_dir, exist_ok=True)

    try:
        with pdfplumber.open(file_path) as pdf:
            page = pdf.pages[0]  # İlk sayfayı al

            for img_data in page.images:
                x0, y0, x1, y1 = img_data['x0'], img_data['y0'], img_data['x1'], img_data['y1']

                # Görsel stream olarak varsa işle
                if 'stream' in img_data:
                    img_bytes = img_data['stream'].get_data()  # Görseli byte olarak al
                    temp_filename = os.path.join(temp_dir, f"img_{len(project_data['images'])}.png")

                    image = Image.open(io.BytesIO(img_bytes))

                    # ReportLab koordinat sistemine uyum için dikey çevir
                    image = image.transpose(Image.FLIP_TOP_BOTTOM)
                    image.save(temp_filename)  # Geçici dosyaya kaydet

                    # Y koordinatını ReportLab sistemine göre ayarla
                    y_reportlab = y0

                    # Görsel bilgilerini proje veri yapısına ekle
                    project_data['images'].append({
                        'x': x0,
                        'y': y_reportlab,
                        'width': x1 - x0,
                        'height': y1 - y0,
                        'file': temp_filename
                    })

    except Exception as e:
        print(f"pdfplumber ile görsel çıkarılırken hata: {e}")


3. Aşama: ReportLab ile Birebir Üretim
Bu fonksiyon, çıkarılan tüm veriyi kullanarak hedef output.pdf dosyasını oluşturur. Output oluşturulurken dikdörtgen şekil üzerinden oluşturulmuştur.Sonrasında sağ ve sol çizgisini kaldırarak esas görünüm elde edilmiştir

In [19]:
# ReportLab Üretimi
def create_output_pdf(output_file):
    """project_data içindeki metin, çizgi, dikdörtgen ve görselleri birebir konumlarıyla yeni bir PDF'e çizer."""

    # Canvas oluştur: sayfa boyutu daha önce get_page_dimensions ile alınmış olan page_w, page_h
    c = canvas.Canvas(output_file, pagesize=(page_w, page_h))

    # Çizgi renklerini siyah olarak ayarla
    c.setStrokeColor(black)


    FIXED_LINE_WIDTH = 1.5  # tüm çizgiler için sabit ve biraz kalın
    c.setLineWidth(FIXED_LINE_WIDTH)

    # --- Tüm çizgi nesneleri (LTLine) ---
    for line in project_data.get('lines', []):
        c.line(line['x0'], line['y0'], line['x1'], line['y1'])

    # --- Tüm dikdörtgenlerden gelen çizgiler (LTRect) ---
    for rect in project_data.get('rects', []):
        x0, y0, x1, y1 = rect['x0'], rect['y0'], rect['x1'], rect['y1']

        # Dikdörtgenin üst ve alt kenarını çizgi gibi çiz
        c.line(x0, y1, x1, y1)  # üst kenar
        c.line(x0, y0, x1, y0)  # alt kenar


    # ---- Görselleri Çizme ----
    for img in project_data.get('images', []):
        try:
            # Daha önce hesaplanan X, Y, width ve height değerleri ile görseli yerleştir
            # mask='auto': şeffaflık varsa uygula
            c.drawImage(
                img['file'],
                img['x'],
                img['y'],  # Y koordinatı zaten dikey çeviri ile düzeltildi
                width=img['width'],
                height=img['height'],
                mask='auto'
            )
        except Exception as e:
            print(f"Görsel çiziminde hata (X: {img['x']:.2f}, Y: {img['y']:.2f}): {e}") #Hata durumunda belirt

    # ---- Metin Bloklarını Çizme ----
    for block in project_data.get('text_blocks', []):
        x, y, font_name, font_size, text = block['x'], block['y'], block['font'], block['size'], block['text']

        # ReportLab için güvenli font ismi seç
        reportlab_font = 'Helvetica'  # Varsayılan font
        normalized_name = font_name.upper()
        # FONT_MAPPING ile PDF'deki fontu ReportLab fontuna eşleştir
        for key, value in FONT_MAPPING.items():
            if key in normalized_name:
                reportlab_font = value
                break

        c.setFont(reportlab_font, font_size)  # Font ve boyutu ayarla
        c.drawString(x, y, text)  # Metni çiz

    # ---- PDF'i Kaydetme ve Geçici Görselleri Temizleme ----
    c.save()  # PDF'i kaydet
    shutil.rmtree('temp_images', ignore_errors=True)  # temp_images klasörünü sil
    return output_file  # Oluşturulan PDF dosya yolunu döndür


4. Sonuç ve Çalıştırma Akışı
Bu son kod bloğu, tüm adımları sırayla çağırır ve sonucu gösterir.

In [20]:
# SONUÇ BLOKU: TÜM SÜRECİ ÇALIŞTIRMA

print("--- BAŞLANGIÇ: file.pdf ANALİZİ ---")

# 1️⃣ PDF sayfa boyutlarını al
if get_page_dimensions(PDF_INPUT_FILE):
    # 2️⃣ Metin, çizgi ve dikdörtgenleri çıkar
    extract_all_structural_elements(PDF_INPUT_FILE)
    # 3️⃣ Görselleri çıkar
    extract_images_with_pdfplumber(PDF_INPUT_FILE)

    # Sayfa boyutları başarılı bir şekilde alındıysa
    if page_w != 0 and page_h != 0:
        print(f"✅ VERİ ÇIKARIMI BAŞARILI.")
        print(f"   Sayfa Boyutları: {page_w:.2f} x {page_h:.2f}")
        print(f"   Metin Bloğu Sayısı: {len(project_data['text_blocks'])}")
        print(f"   Görsel Sayısı (Yıldızlar): {len(project_data['images'])}")

        print("\n--- REPORTLAB İLE BİREBİR ÜRETİM BAŞLIYOR ---")
        try:
            # 4️⃣ Extract edilen verilerle birebir PDF üret
            output_file_name = create_output_pdf(PDF_OUTPUT_FILE)

            print(f"\n🏆 PROJE TAMAMLANDI: '{output_file_name}' başarıyla üretildi.")
            print("Lütfen oluşan 'output.pdf' dosyasını orijinali ile görsel olarak karşılaştırın.")

            # 5️⃣ Colab ortamında çıktı dosyasını indirmek için
            from google.colab import files
            files.download(output_file_name)

        except Exception as e:
            # PDF üretimi sırasında kritik hata olursa bildir
            print(f"❌ HATA: PDF üretimi sırasında kritik hata: {e}")
    else:
        # Sayfa boyutları alınamadıysa uyarı
        print("❌ HATA: Boyutlar alınamadı.")
else:
    # PDF dosyası bulunamadı veya açılamadıysa uyarı
    print("❌ HATA: Dosya bulunamadı veya okunamadı. Lütfen 'file.pdf'i yükleyin.")


--- BAŞLANGIÇ: file.pdf ANALİZİ ---
✅ VERİ ÇIKARIMI BAŞARILI.
   Sayfa Boyutları: 595.50 x 842.25
   Metin Bloğu Sayısı: 41
   Görsel Sayısı (Yıldızlar): 2

--- REPORTLAB İLE BİREBİR ÜRETİM BAŞLIYOR ---

🏆 PROJE TAMAMLANDI: 'output.pdf' başarıyla üretildi.
Lütfen oluşan 'output.pdf' dosyasını orijinali ile görsel olarak karşılaştırın.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>